In [11]:
## Nipype Workflow for Erin's DTI Data
### Rename and Reorganize  NIFTI Image Data set
import os, sys
from os.path import join as oj
from glob import glob
import shutil

import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
from nipype.caching import Memory
from nipype.interfaces.dcm2nii import Dcm2nii

In [12]:
from StoutSubjectDict import StoutSubjectIDMap
import StoutData_ConvertRawData as scrd
from pprint import pprint

StoutRawData = '/home/ehecht/BIGDATA/Stout_Homo_Faber/RAWDATA/'
RawNIIDir = '/EINSTEIN_BIGDATA/NIPYPE_WD/Stout_Haber/niiData_V4/'
NiPypeOutputDir = '/EINSTEIN_BIGDATA/NIPYPE_WD/Stout_Haber/niPypePreProc/'
mem = Memory(base_dir=NiPypeOutputDir)  ## Create a memory cache I can use going forward

dcmConverter = mem.cache(Dcm2nii)


In [13]:
from ipyparallel import Client
rc = Client()
dview = rc[:]
dview.block=False
#!conda install ipyparallel --yes

In [14]:
dview.targets

[0, 1, 2, 3, 4, 5, 6, 7]

In [6]:
# reload( StoutSubjectDict)
reload(scrd)

### Zip up all the data so the subjID and folder are easier to keep track of
scanImageData  = []
for curScanSession in StoutSubjectIDMap:
    curScanSession['dcmScanSet'] =  scrd.DicomScanSet
    curScanSession['dcmRoot'] =  scrd.StoutRawData


scanImageData = StoutSubjectIDMap    
print len(scanImageData),'Scan Image Data Sets have been located'

81 Scan Image Data Sets have been located


In [ ]:
HomoFaberImageSessions = [] ### Null Array this will have the image sessions that have an actual Directory
                            ### Helps me avoid typos
UnknownImageSession = []

MappedFolders = [x['folder'] for x in StoutSubjectIDMap]  ## I get this list from a module
foldersToIgnore = ['test_BITC20141114','test_BITC20141118'] 

for ss in os.listdir(StoutRawData):
    fullSubjDirPath =  oj(StoutRawData,ss)
    if ss not in MappedFolders and ss not in foldersToIgnore:
        #print "Could not find %s in subject ID list" % ss
        UnknownImageSession.append(ss)
    elif os.path.isdir(fullSubjDirPath):
        HomoFaberImageSessions.append(ss)
        
# print len(HomoFaberImageSessions),"Subjects to process"

In [ ]:
# from random import shuffle
# shuffle(scanImageData)

print len(UnknownImageSession)
print len(StoutSubjectIDMap)
for sf in UnknownImageSession:
    pass
    #print "{'folder': '%s', 'scanID': ' ', 'sessionNum': ' '}" % sf

In [ ]:
# for imgSessionDict in scanImageData:
#     ### To make this work easier, I am passing a single dictionary with all the params I need
#     ### it's very confusing how to pass more than one parameter at a time using the MAP function
#     CheckCompleteDataSet(DicomScanSet, imgSessionDict,  StoutRawData,debug=False)

In [ ]:
print len(scanImageData)

In [ ]:
renamingDict = [ { 'inputDir': 'T1/co*.nii.gz', 'outputFileWPath': 'T1/struc_raw.nii.gz', 'fileType': 'T1'},
                 { 'inputDir': 'DTI/preprocess/diff5B0_PA/*.nii.gz', 'outputFileWPath': 'DTI/preprocess/5B0_PA/nodif_PA.nii.gz', 'fileType': 'nodiff_PA'},
                 { 'inputDir': 'DTI/preprocess/diff5B0_PA/*.bval', 'outputFileWPath': 'DTI/preprocess/5B0_PA/nodif_PA_bval', 'fileType': 'nodiff_PA_bval'},
                 { 'inputDir': 'DTI/preprocess/diff5B0_PA/*.bvec', 'outputFileWPath': 'DTI/preprocess/5B0_PA/nodif_PA_bvec', 'fileType': 'nodiff_PA_bvec'},
                 { 'inputDir': 'DTI/preprocess/AP/*.nii.gz', 'outputFileWPath': 'DTI/preprocess/AP/data_AP.nii.gz', 'fileType': 'data_AP'},
                 { 'inputDir': 'DTI/preprocess/AP/*.bval', 'outputFileWPath': 'DTI/preprocess/AP/data_AP_bval', 'fileType': 'data_AP_bval'},
                 { 'inputDir': 'DTI/preprocess/AP/*.bvec', 'outputFileWPath': 'DTI/preprocess/AP/data_AP_bvec', 'fileType': 'data_AP_bvec'},
                ]

CompleteNIISetDir = '/FEATURES/NiPypeWorkingData/StoutHaber/NII_Data_Complete'

#print sorted(os.listdir(RawNIIDir))
for ImgSession in sorted(os.listdir(RawNIIDir)):
    print ImgSession,
    ###I NEED TO DETERMINE IF A RawNIIDir contains all of the necessary files
    foundCompleteSet = True
    for ftrD in renamingDict: #file to rename Dictionary
        g= glob( oj( RawNIIDir, ImgSession, ftrD['inputDir']))
        if len(g) != 1:
            foundCompleteSet = False
            print "No Match", ftrD['inputDir']
    if foundCompleteSet:
        print "***COMPLETE***"
        moveCompleteDataSet( ImgSession, CompleteNIISetDir)
    else:
        print "XXX INCOMPLETE XXX"


In [7]:
#def CheckCompleteDataSet( dataSetDict, imageSessionDict, dcmRootDir, debug=False):
@dview.parallel(block=False)    
def CheckCompleteDataSet( processingInfoDict):
    
    """This expects a dictionary which has the primary key as the image type, that then yields another
    dictionary of the dcmDir and outputDirName that should be associated with the input type, see example
    above"""
    debug=False
    pid = processingInfoDict  ## made an alias
    
    foundCompleteDCMSet = True
    if debug: print "looking for files for %s " %  pid['folder']
    ### I will still try and convert incomplete sets, at least for now
    
    for imageType in pid['dcmScanSet']:
        if debug: print "looking for %s" % imageType
        dcmDirPath = oj(pid['dcmRoot'], pid['folder'],pid['dcmScanSet'][imageType]['dcmDir']) ### There should be only one of these
        imgDcmDir =  glob(dcmDirPath)
        if len(imgDcmDir) > 1: 
            print "Found more than one matching directory"
            print imgDcmDir
            #sys.exit
        elif len(imgDcmDir) == 0:
            print "Not a complete set",dcmDirPath
            foundCompleteDCMSet = False
        else:
            #### This means I found one and only 1 match, so I will not convert it's contents and then move the output

            
            NIIOutputDir = oj( RawNIIDir, pid['scanID'], pid['dcmScanSet'][imageType]['outputDirName'])
                        ### If the target directory already exists It means I already did this one so scan skip
            if not os.path.isdir(NIIOutputDir):
                generateNIIData( imgDcmDir[0], imageType,  NIIOutputDir,debug=True)


In [ ]:

def moveCompleteDataSet( ImgSessionName, TargetDir, overWriteExisting=False):
    """Once I know I have a complete set of NII images, I am going to move them to a target directory for further processing"""
#     print ImgSessionName,"Needs to be moved"
    ## Walk the renaming dict and move the appropriate files
    for ftrD in renamingDict: #file to rename Dictionary
        inputFile = glob( oj( RawNIIDir, ImgSession, ftrD['inputDir']))[0]
        TargetFileNameWPath=  oj( TargetDir, ImgSessionName, ftrD['outputFileWPath'])
#         print inputFile,TargetFileNameWPath
        tgtDir = os.path.dirname(TargetFileNameWPath)
        if not os.path.isdir(tgtDir):
            os.makedirs(tgtDir)
        if not os.path.isfile(TargetFileNameWPath):
            shutil.copy(inputFile,TargetFileNameWPath)
            
        


In [10]:
print len(scanImageData)

#print scanImageData[2]
JobStatus = CheckCompleteDataSet.map(scanImageData)

81


In [ ]:
               
# $statement  = " mv $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP_ADC/*.nii.gz ";
# $statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP_ADC/AP_ADC.nii.gz ";
# #print "$statement \n";
# $statement  = " mv $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP_FA/*.nii.gz ";
# $statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP_FA/AP_FA.nii.gz ";



In [ ]:
JobStatus.result()

In [ ]:
print scanImageData

In [ ]:

# #print "$statement \n";
# $statement  = " mv $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP_TRACEW/*.nii.gz ";
# $statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP_TRACEW/AP_TRACEW.nii.gz ";
# #print "$statement \n";
# $statement  = " mv $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP_TRACEW/*.bval ";
# $statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP_TRACEW/AP_TRACEW_bval ";
# #print "$statement \n";
# $statement  = " mv $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP_TRACEW/*.bvec ";
# $statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP_TRACEW/AP_TRACEW_bvec ";
# #print "$statement \n";

In [ ]:
JobStatus.running()

In [9]:
%px from os.path import join as oj
%px import numpy 
%px cd /home/dgutman/devel/GutmanLabNipypeWorkflows/IpythonNotebooks

%px from StoutData_ConvertRawData import generateNIIData
%px from glob import glob
%px RawNIIDir = '/EINSTEIN_BIGDATA/NIPYPE_WD/Stout_Haber/niiData_V4'
%px import os


[stdout:0] /home/dgutman/devel/GutmanLabNipypeWorkflows/IpythonNotebooks
[stdout:1] /home/dgutman/devel/GutmanLabNipypeWorkflows/IpythonNotebooks
[stdout:2] /home/dgutman/devel/GutmanLabNipypeWorkflows/IpythonNotebooks
[stdout:3] /home/dgutman/devel/GutmanLabNipypeWorkflows/IpythonNotebooks
[stdout:4] /home/dgutman/devel/GutmanLabNipypeWorkflows/IpythonNotebooks
[stdout:5] /home/dgutman/devel/GutmanLabNipypeWorkflows/IpythonNotebooks
[stdout:6] /home/dgutman/devel/GutmanLabNipypeWorkflows/IpythonNotebooks
[stdout:7] /home/dgutman/devel/GutmanLabNipypeWorkflows/IpythonNotebooks
